Jupyter notebook waarmee de pygeoapi demo server benadert kan worden om de logboek dataverwerking implementatie te demonstreren.

De opentelemetry logs worden in te terminal getoond waar docker compose is uitgevoerd.

In [1]:
import folium
import requests  # For making HTTP requests
import geopandas as gpd
import json

In [2]:

def handle_response(response):
  if response.status_code == 200:
    
    return response.json()
    
  else:
    print(f"Error: {response.status_code, response.json()['description']}")

Onderstaande code roept een OGC API Process functie aan die een getal vermenigvuld met zichzelf.

Het resultaat van de functie wordt hier getoond. In de terminal wordt de opentelemery log geschreven van deze actie.

In [7]:
def calculate_and_display(number):
  url = "http://localhost:5000/processes/squared/execution"  
  data = {"inputs": {"number-or-integer": number}}
  headers = {"Content-Type": "application/json"}

  response = requests.post(url, json=data, headers=headers)
  return handle_response(response)

# Example usage
# user_input = int(input("Enter a number: "))
user_input = 42
result = calculate_and_display(user_input)

print(result)


{'id': 'squared', 'value': 1764}


De andere OGC API process functie maakt gebruik van een OGC API Features dataset. Deze dataset wordt hier op een kaart getoond

In [4]:
m = folium.Map(location=[52.2, 5.5], zoom_start=8)

url = "http://localhost:5000/collections/knmi_meetstations/items?f=json&limit=1000"

# Fetch the GeoJSON data
gdf = gpd.read_file(url)

#quick hack omdat de datetime kolom niet goed naar geojson geconverteerd wordt...
gdf_subset = gdf[['STN','TYPE', 'geometry']]

# Convert to GeoJSON
geojson = gdf_subset.to_json()

folium.GeoJson(
    gdf_subset,
    name="Stations",
    style_function=lambda x: {'fillColor': 'blue', 'color': 'black', 'weight': 1, 'fillOpacity': 0.5},
    tooltip=folium.GeoJsonTooltip(fields=["STN"]),
    popup=folium.GeoJsonPopup(fields=["STN", "TYPE"]),
    highlight_function=lambda x: {"fillOpacity": 0.8},
).add_to(m)

m

Deze OGC API Process functie gebruikt een [scikit-learn unsupervised anomalie detectie algoritme](https://scikit-learn.org/stable/auto_examples/neighbors/plot_lof_outlier_detection.html). In dit scenario willen we bepalen welke KNMI Waarneemstations als 'outlier' beschouwt kunnen worden (omdat ze te ver weg liggen) en niet meegenomen moeten worden in eventuele vervolg berekeningen. Dit is een puur hypothetisch voorbeeld om de functionaliteit van Logboek Dataverwerking voor Geo-objecten te laten zien.

- input: een OGC API Features dataset (url)
- output: een geojson file waar een extra attribuut aan is toegevoegd, 'abnormality' met de waarde 1 of -1.

In [6]:
def calc_lof(feat):
    url = "http://localhost:5000/processes/localoutlier_simple/execution"  
    data = {"inputs": {"dataset": feat}}
    headers = {"Content-Type": "application/json"}

    response = requests.post(url, json=data, headers=headers)
    return handle_response(response)


#let op: omdat dit de url is vanaf de ogc api process functie naar de ogc api features op dezelfde server is de url zonder poort 5000
input = 'http://localhost/collections/knmi_meetstations/items?f=json&limit=1000' 
result = calc_lof(input)

print(result)
with open('meetstations.geojson', 'w') as f:
    json.dump(json.loads(result['value']), f)

{'id': 'output_dataset', 'value': '{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"STN": 201, "TYPE": "Platform/AWS", "abnormality": -1}, "geometry": {"type": "Point", "coordinates": [2.9358, 54.3256]}}, {"id": "1", "type": "Feature", "properties": {"STN": 203, "TYPE": "Platform/AWS", "abnormality": 1}, "geometry": {"type": "Point", "coordinates": [3.3417, 52.36]}}, {"id": "2", "type": "Feature", "properties": {"STN": 204, "TYPE": "Platform/AWS", "abnormality": 1}, "geometry": {"type": "Point", "coordinates": [3.6278, 53.2694]}}, {"id": "3", "type": "Feature", "properties": {"STN": 205, "TYPE": "Platform/AWS", "abnormality": -1}, "geometry": {"type": "Point", "coordinates": [3.8103, 55.3992]}}, {"id": "4", "type": "Feature", "properties": {"STN": 206, "TYPE": "Platform/AWS", "abnormality": -1}, "geometry": {"type": "Point", "coordinates": [4.0122, 54.1167]}}, {"id": "5", "type": "Feature", "properties": {"STN": 207, "TYPE": "Platform/AWS", "abn

In [21]:
m = folium.Map(location=[52.2, 5.5], zoom_start=8)
gdf = gpd.read_file('meetstations.geojson')
gdf2 = gdf[['STN','TYPE', 'geometry','abnormality']]


folium.GeoJson(
    gdf2,
    name="Stations",
    marker=folium.Marker(icon=folium.Icon(icon='star')),
    tooltip=folium.GeoJsonTooltip(fields=["STN","abnormality"]),
    popup=folium.GeoJsonPopup(fields=["STN", "TYPE"]),
    style_function=lambda x: {
        'markerColor': 'orange' if x['properties']['abnormality'] == -1 else 'green',
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0.7
    },
    highlight_function=lambda x: {"fillOpacity": 0.8},
).add_to(m)

value_counts = gdf['abnormality'].value_counts()
print(value_counts)
m

abnormality
 1    67
-1    10
Name: count, dtype: int64


In [34]:
def calc_lof(feat):
    url = "http://localhost:5000/processes/localoutlier_simple/execution"  
    data = {"inputs": {"dataset": feat}}
    headers = {"Content-Type": "application/json"}

    response = requests.post(url, json=data, headers=headers)
    return handle_response(response)


#let op: omdat dit de url is vanaf de ogc api process functie naar de ogc api features op dezelfde server is de url zonder poort 5000
input = 'http://localhost/collections/knmi_meetstations/items?f=json&limit=1000' 
result = calc_lof(input)

print(result)
with open('meetstations.geojson', 'w') as f:
    json.dump(json.loads(result['value']), f)

{'id': 'output_dataset', 'value': '{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"STN": 201, "TYPE": "Platform/AWS", "abnormality": -1}, "geometry": {"type": "Point", "coordinates": [2.9358, 54.3256]}}, {"id": "1", "type": "Feature", "properties": {"STN": 203, "TYPE": "Platform/AWS", "abnormality": 1}, "geometry": {"type": "Point", "coordinates": [3.3417, 52.36]}}, {"id": "2", "type": "Feature", "properties": {"STN": 204, "TYPE": "Platform/AWS", "abnormality": 1}, "geometry": {"type": "Point", "coordinates": [3.6278, 53.2694]}}, {"id": "3", "type": "Feature", "properties": {"STN": 205, "TYPE": "Platform/AWS", "abnormality": -1}, "geometry": {"type": "Point", "coordinates": [3.8103, 55.3992]}}, {"id": "4", "type": "Feature", "properties": {"STN": 206, "TYPE": "Platform/AWS", "abnormality": -1}, "geometry": {"type": "Point", "coordinates": [4.0122, 54.1167]}}, {"id": "5", "type": "Feature", "properties": {"STN": 207, "TYPE": "Platform/AWS", "abn